In [7]:
import numpy as np
import pandas as pd
import requests 
import json
from pandas.io.json import json_normalize


# URL
# https://waterdata.usgs.gov/pa/nwis/uv?format=rdb&site_no=01571500&period=&begin_date=2021-01-01&end_date=2021-01-20


In [8]:
BASE_URL = 'https://nwis.waterdata.usgs.gov/pa/nwis/uv/'
FORMAT = 'json' # rdb
SITE_NUMBER = '01571500' # Yellow Breeches Creek near Camp Hill, PA
BEGIN_DATE = '2020-01-01' # 1985-10-01 / 2007-10-01
END_DATE = '2020-01-15' # 2021-01-18 / 2021-01-18

In [11]:
# url = 'https://nwis.waterdata.usgs.gov/pa/nwis/uv/?format={}&site_no={}&period=&begin_date={}&end_date={}'.format(FORMAT, SITE_NUMBER, BEGIN_DATE, END_DATE)
url = 'https://nwis.waterservices.usgs.gov/nwis/iv/?site=01571500&format=json&startDT=2020-01-01&endDT=2020-02-01'
url

'https://nwis.waterservices.usgs.gov/nwis/iv/?site=01571500&format=json&startDT=2020-01-01&endDT=2020-02-01'

In [12]:
# Get Request
response = requests.get(url).json()
response
# # response

# file1 = open("data/USGS/Yellow-Breeches-Creek/raw.txt","w")#write mode 
# file1.write(response) 
# file1.close() 

{'name': 'ns1:timeSeriesResponseType',
 'declaredType': 'org.cuahsi.waterml.TimeSeriesResponseType',
 'scope': 'javax.xml.bind.JAXBElement$GlobalScope',
 'value': {'queryInfo': {'queryURL': 'http://nwis.waterservices.usgs.gov/nwis/iv/site=01571500&format=json&startDT=2020-01-01&endDT=2020-02-01',
   'criteria': {'locationParam': '[ALL:01571500]',
    'variableParam': 'ALL',
    'timeParam': {'beginDateTime': '2020-01-01T00:00:00.000',
     'endDateTime': '2020-02-01T23:59:59.000'},
    'parameter': []},
   'note': [{'value': '[ALL:01571500]', 'title': 'filter:sites'},
    {'value': '[mode=RANGE, modifiedSince=null] interval={INTERVAL[2020-01-01T00:00:00.000-05:00/2020-02-01T23:59:59.000Z]}',
     'title': 'filter:timeRange'},
    {'value': 'methodIds=[ALL]', 'title': 'filter:methodId'},
    {'value': '2021-01-20T04:51:01.479Z', 'title': 'requestDT'},
    {'value': '182b9860-5adb-11eb-952b-4cd98f8df011', 'title': 'requestId'},
    {'value': 'Provisional data are subject to revision. Go 

In [46]:
foobar = response['value']['timeSeries'][0]['variable']
foobar

{'variableCode': [{'value': '00060',
   'network': 'NWIS',
   'vocabulary': 'NWIS:UnitValues',
   'variableID': 45807197,
   'default': True}],
 'variableName': 'Streamflow, ft&#179;/s',
 'variableDescription': 'Discharge, cubic feet per second',
 'valueType': 'Derived Value',
 'unit': {'unitCode': 'ft3/s'},
 'options': {'option': [{'name': 'Statistic', 'optionCode': '00000'}]},
 'note': [],
 'noDataValue': -999999.0,
 'variableProperty': [],
 'oid': '45807197'}

In [73]:
for i, series in enumerate(response['value']['timeSeries']):
    print("Series: ", str(i))
    print("Variable Name: ", series['variable']['variableName'])
    print("Variable Description: ", series['variable']['variableDescription'])
    print("Variable Units: ", series['variable']['unit']['unitCode'])
    
    # Create Data Frame
    df = json_normalize(series['values'][0]['value'])
    
    # Set Index to Date Time
    df.set_index('dateTime', inplace=True)
    # Remove Index Name
    df.index.name = None
    # Rename Value Column to be the Variable Name
    new_value_name = series['variable']['variableName'].split(',')[0].replace(' ', '_').lower()
    df.rename(columns={'value': new_value_name}, inplace=True)
    # Add Unit Column
    df[new_value_name+'_units'] = series['variable']['unit']['unitCode']
    # Drop Qualifiers Columns
    df.drop(['qualifiers'], axis=1, inplace=True)
    
    print(df.head())
    print('\n')

Series:  0
Variable Name:  Streamflow, ft&#179;/s
Variable Description:  Discharge, cubic feet per second
Variable Units:  ft3/s
                              streamflow streamflow_units
2020-01-01T00:00:00.000-05:00        311            ft3/s
2020-01-01T00:15:00.000-05:00        311            ft3/s
2020-01-01T00:30:00.000-05:00        308            ft3/s
2020-01-01T00:45:00.000-05:00        311            ft3/s
2020-01-01T01:00:00.000-05:00        308            ft3/s


Series:  1
Variable Name:  Gage height, ft
Variable Description:  Gage height, feet
Variable Units:  ft
                              gage_height gage_height_units
2020-01-01T00:00:00.000-05:00        1.69                ft
2020-01-01T00:15:00.000-05:00        1.69                ft
2020-01-01T00:30:00.000-05:00        1.68                ft
2020-01-01T00:45:00.000-05:00        1.69                ft
2020-01-01T01:00:00.000-05:00        1.68                ft




<ipython-input-73-47555f041c57>:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(series['values'][0]['value'])
<ipython-input-73-47555f041c57>:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(series['values'][0]['value'])


In [43]:
value = response['value']['timeSeries'][0]['values'][0]['value']
value

[{'value': '311',
  'qualifiers': ['A'],
  'dateTime': '2020-01-01T00:00:00.000-05:00'},
 {'value': '311',
  'qualifiers': ['A'],
  'dateTime': '2020-01-01T00:15:00.000-05:00'},
 {'value': '308',
  'qualifiers': ['A'],
  'dateTime': '2020-01-01T00:30:00.000-05:00'},
 {'value': '311',
  'qualifiers': ['A'],
  'dateTime': '2020-01-01T00:45:00.000-05:00'},
 {'value': '308',
  'qualifiers': ['A'],
  'dateTime': '2020-01-01T01:00:00.000-05:00'},
 {'value': '308',
  'qualifiers': ['A'],
  'dateTime': '2020-01-01T01:15:00.000-05:00'},
 {'value': '306',
  'qualifiers': ['A'],
  'dateTime': '2020-01-01T01:30:00.000-05:00'},
 {'value': '306',
  'qualifiers': ['A'],
  'dateTime': '2020-01-01T01:45:00.000-05:00'},
 {'value': '308',
  'qualifiers': ['A'],
  'dateTime': '2020-01-01T02:00:00.000-05:00'},
 {'value': '306',
  'qualifiers': ['A'],
  'dateTime': '2020-01-01T02:15:00.000-05:00'},
 {'value': '306',
  'qualifiers': ['A'],
  'dateTime': '2020-01-01T02:30:00.000-05:00'},
 {'value': '306',
  '